In [ ]:
import nltk

In [ ]:
sentences = "The cat is in the box. The cat likes the box. The box is over the cat."

## 1. Preprocessing

### A. Tokenizacja

In [ ]:
nltk.download('punkt')

In [ ]:
# tokenizacja
from nltk.tokenize import word_tokenize

tokens = word_tokenize(sentences)
print(tokens)

### B. Lowercase

In [ ]:
tokens = [token.lower() for token in tokens]
print(tokens)

### C. Usunięcie niealfanumerycznych tokenów

In [ ]:
tokens = [token for token in tokens if token.isalpha()]
print(tokens)

In [ ]:
# gdybyśmy chcieli usunąć wyłącznie znaki, które nie są interpunkcyjne, a nie wszystkie niealfanumeryczne znaki, możemy zrobić tak
import string

tokens = [token for token in tokens if token not in string.punctuation]
print(tokens)

### D. Usunięcie stopwords

**Stopwords** - najczęściej występujące słowa w języku, które nie niosą ze sobą żadnej konkretnej treści.

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords

stopwords_list = stopwords.words('english')
print(stopwords_list)

In [ ]:
tokens = [token for token in tokens if token not in stopwords_list]
print(tokens)

### E. Lematyzacja

In [ ]:
nltk.download('wordnet')

In [ ]:
from nltk.stem import WordNetLemmatizer

# Inicjalizacja WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

tokens = [wordnet_lemmatizer.lemmatize(token) for token in tokens]
print(tokens)

## 2. Techniki osadzania słów, *ang. embeddings* (aka wektoryzacja słów)

### A. Model BoW

In [ ]:
from collections import Counter

c = Counter(tokens)
print(c)

In [ ]:
# dwa najczęściej występujące tokeny w tekście
print(c.most_common(2))

Zróbmy teraz to samo wykorzystując inną bibliotekę.

#### Biblioteka Gensim

Biblioteka Gesim jest drugą, popularną biblioteką do przetwarzania tekstu w pythonie. Pozwala nam w prosty sposób budować korpusy i słowniki. **Korpus** to zbiór tekstów, na którym wykonujemy zadania przetwarzanie języka naturalnego. Składa się z pojedynczych **dokumentów**. Zbiór słów występujących w korpusie nazywamy jego **słownikiem**.

In [ ]:
!pip install gensim

Najpier tokenizacja i oczyszcenie tekstu. Do tych zadań użyjemy nltk

In [ ]:
from nltk.tokenize import word_tokenize

# nasz dokument (lista napisów)
my_documents = [
    'Nearly all great ideas follow a similar creative process and this article explains how this process works. Understanding this is important because creative thinking is one of the most useful skills you can possess.',
    'Not doing something will always be faster than doing it. This statement reminds me of the old computer programming saying, Remember that there is no code faster than no code.',
    'He went on to become a trailblazer in the field of photography and held over 70 patents by the end of his career. His story of creativity and innovation, which I will share now, is a useful case study for understanding the 5 key steps of the creative process.',
    'He spent the rest of the decade experimenting with new photography techniques and learning about cameras, printers, and optics.',   
]

# preprocessing (lowercase, usuwam znaki interpunkcyjne i stopwords)
tokenized_docs = [word_tokenize(article.lower()) for article in my_documents]
for idx, article_word_list in enumerate(tokenized_docs):
    new_article_word_list = []
    for token in article_word_list:
        if (not token.isalpha()) or token in stopwords_list:
            continue
        else:
            new_article_word_list.append(token)
    tokenized_docs[idx]=new_article_word_list

print(tokenized_docs)

Dla tak przygotowanych danych tekstowych wygnerujmy model bow za pomocą narzędzi biblioteki Gensim.  

Stworzenie słownika - czyli jednoznaczne przyporządkowanie każdemu tokenowi jakiejś liczby

In [ ]:
from gensim.corpora.dictionary import Dictionary

# Słownik (mapa)
dictionary = Dictionary(tokenized_docs)
print(dictionary.token2id)

Słownik posiada metodę doc2bow, która zwraca rozkład bow przekazanego do niej dokumentu. Zliczane są wyłącznie tokeny występujące w słowniku. Wynikowa lista składa się z dwuelementowych tupli, w których pierwszy element to id tokena w słwoniku, a drugi element to liczba wystąpień tego tokena w przekazanym do metody dokumentcie.

In [ ]:
bow = dictionary.doc2bow(tokenized_docs[0])
print(bow)

W ten sposób możemy otrzymać rozkład bow dla całego naszego korpusu.

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]
print(bow_corpus)

### B. Model TF-IDF

In [ ]:
from gensim.models.tfidfmodel import TfidfModel

tfidf_corpus = TfidfModel(bow_corpus)

In [ ]:
print(bow_corpus[0])

In [ ]:
print(tfidf_corpus[bow_corpus[0]])

Mimo, że token o id 15 występuje w dokumencie 1 raz, czyli dokładnie tyle samo razy co tokeny o id 1,2,3,..., to wartość przyporządkowana jemu w tfidf jest niższa. Widocznie ten token występuje częściej w innych dokumentach od pozostałych tokenów pierwszego dokumentu.

In [ ]:
dictionary[15]